In [ ]:
!pip install dgl
!pip install torch==1.9.1
!pip install node2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 17.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement torch==1.9.1 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1)
ERROR: No matching distribution found for torch==1.9.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.4 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.2.1
    Uninstalling networkx-3.2.1:
      Successfully uninstalled networkx-3.2.1


In [ ]:
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import SAGEConv,GCN2Conv
import dgl.function as fn
import itertools
import numpy as np
import scipy.sparse as sp
import sklearn.metrics
from node2vec import Node2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score,roc_curve

from dgl.nn import GraphConv

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


load CORA dataset

In [ ]:
dataset = dgl.data.CoraGraphDataset()
# dataset = dgl.data.CiteseerGraphDataset()
# dataset = dgl.data.PubmedGraphDataset()
g = dataset[0]

Extracting file to /root/.dgl/cora_v2_d697a464
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.




---


edges of CORA graph


---



In [ ]:
u, v = g.edges()

In [ ]:
edge_ids = np.arange(g.number_of_edges())
edge_ids = np.random.permutation(edge_ids)
test_size = int(len(edge_ids) * 0.1)
train_size = g.number_of_edges() - test_size
test_pos_u, test_pos_v = u[edge_ids[:test_size]], v[edge_ids[:test_size]]
train_pos_u, train_pos_v = u[edge_ids[test_size:]], v[edge_ids[test_size:]]

In [ ]:
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_edge_ids = np.random.choice(len(neg_u), g.number_of_edges() // 2)
test_neg_u, test_neg_v = neg_u[neg_edge_ids[:test_size]], neg_v[neg_edge_ids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_edge_ids[test_size:]], neg_v[neg_edge_ids[test_size:]]



---

adjacency matrix of CORA


---



In [ ]:
adj.shape

(2708, 2708)

In [ ]:
train_g = dgl.remove_edges(g, edge_ids[:test_size])

In [ ]:
train_g = dgl.add_self_loop(train_g)

In [ ]:
def get_optimal_threshold(tpr, fpr, thresholds):
  g_means = tpr*(1-fpr)
  max_index = np.argmax(g_means)
  return thresholds[max_index]

In [ ]:
def binarize_scores(scores, labels, optimal_threshold):
  final_preds = []
  for i in range(len(scores)):
    if(scores[i]<=optimal_threshold):
      final_preds.append(0)
    else:
      final_preds.append(1)
  return np.asarray(final_preds)

In [ ]:
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, h_feats)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


In [ ]:
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

In [ ]:
class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            #computing score
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            return g.edata['score'][:, 0]

In [ ]:
A=adj
s=h

In [ ]:
import random
#-----------calcuate similarity between two embedding vectors
def cal_simi(s1,s2):
  dot_product=torch.dot(s1,s2)
  norm_product=torch.norm(s1)*torch.norm(s2)
  simi=torch.exp(dot_product / norm_product)
  return simi

#---------------------- calculate U+ -----------------
def Uplus(A,s):
  Uplus=[]
  adj=[]
  random_adj=[]
  for i in range(len(A)):
      U=0
      adj=[]
      random_adj=[]
      for j in A[i].nonzero():
        adj.append(j)
      if len(adj)>= 5:
        random_adj.append(random.sample(adj,5))
        random_adj=random_adj[0]
      else:
        random_adj=adj
      if len(random_adj)>0:
        for j in random_adj:
          # print(i,j)
          U+=A[i][j]*cal_simi(s[i],s[j][0])
      Uplus.append(U)
  return Uplus

In [ ]:
import torch
A = A.todense()
A=torch.tensor(A)
Ak=torch.mm(A,A)
Abar=1-Ak
Abar[0]

tensor([-2.,  1.,  1.,  ...,  1.,  1.,  1.], dtype=torch.float64)

In [ ]:
#------------ calculate U- ---------------------
def Umin(A,s):
  U=0
  Umin=[]
  n_adj=[]
  random_adj=[]
  for i in range(len(A)):
    U=0
    for j in torch.nonzero(A[i] == 0):
        n_adj.append(j)
    for j in torch.nonzero(Ak[i] == 0):
      if j !=i:
        n_adj.append(j)
    if len(n_adj)>= 10:
        random_adj=random.sample(n_adj,10)
        random_adj=random_adj[0]
    else:
        random_adj=n_adj
        random_adj=random_adj[0]
    if len(random_adj)>0:
        for j in random_adj:
          # print(i,j)
          if s[j].shape ==torch.Size([1, 2]):
            U+=Abar[i][j]*cal_simi(s[i],s[j][0])
          else:
            U+=Abar[i][j]*cal_simi(s[i],s[j])
    Umin.append(U)
  return Umin

In [ ]:
model = GCN(train_g.ndata['feat'].shape[1], 128)
pred = DotPredictor()

def compute_loss(h, pos_score, neg_score):
    uplus=Uplus(A,h)
    umin=Umin(A,h)
    uplus=torch.tensor(uplus)
    umin=torch.tensor(umin)
    loss=torch.mean(-1*(torch.log(uplus)-torch.log(uplus+umin)))
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return (F.binary_cross_entropy_with_logits(scores, labels)+loss)/2

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores),scores,labels, roc_curve(labels, scores)

In [ ]:
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

all_logits = []
for e in range(100):
    h = model(train_g, train_g.ndata['feat'])
    print("h=",h.shape)
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(h,pos_score, neg_score)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print('In epoch ',e,' loss: ', loss)

h= torch.Size([2708, 128])
In epoch  0  loss:  tensor(0.4680, dtype=torch.float64, grad_fn=<DivBackward0>)
h= torch.Size([2708, 128])
h= torch.Size([2708, 128])
h= torch.Size([2708, 128])
h= torch.Size([2708, 128])
h= torch.Size([2708, 128])
In epoch  5  loss:  tensor(0.4688, dtype=torch.float64, grad_fn=<DivBackward0>)
h= torch.Size([2708, 128])
h= torch.Size([2708, 128])
h= torch.Size([2708, 128])
h= torch.Size([2708, 128])
h= torch.Size([2708, 128])
In epoch  10  loss:  tensor(0.4506, dtype=torch.float64, grad_fn=<DivBackward0>)
h= torch.Size([2708, 128])
h= torch.Size([2708, 128])
h= torch.Size([2708, 128])
h= torch.Size([2708, 128])
h= torch.Size([2708, 128])
In epoch  15  loss:  tensor(0.4485, dtype=torch.float64, grad_fn=<DivBackward0>)
h= torch.Size([2708, 128])
h= torch.Size([2708, 128])
h= torch.Size([2708, 128])
h= torch.Size([2708, 128])
h= torch.Size([2708, 128])
In epoch  20  loss:  tensor(0.4415, dtype=torch.float64, grad_fn=<DivBackward0>)
h= torch.Size([2708, 128])
h= 

KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    auc_score, scores, labels, (tpr, fpr, thresholds) = compute_auc(pos_score, neg_score)
    opt_threshold = get_optimal_threshold(tpr, fpr, thresholds)
    binarized_preds = binarize_scores(scores,labels, opt_threshold)
    print('AUC', auc_score)
    print("accuracy: ",sklearn.metrics.accuracy_score(labels,binarized_preds))
    print("F-measure: ",sklearn.metrics.f1_score(labels,binarized_preds))
    print("Precision: ",sklearn.metrics.precision_score(labels,binarized_preds) )
    print("Recall: ",sklearn.metrics.recall_score(labels,binarized_preds))

AUC 0.8943114485299072
accuracy:  0.7971563981042654
F-measure:  0.7857857857857858
Precision:  0.8324496288441146
Recall:  0.7440758293838863
